In [4]:
# Cell 1: Install dependencies
!pip install opencv-python dlib face_recognition scikit-learn matplotlib

  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
Failed to build dlib


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [41 lines of output]
  running bdist_wheel
  running build
  running build_ext
  
  
                     CMake is not installed on your system!
  
      Or it is possible some broken copy of cmake is installed on your system.
      It is unfortunately very common for python package managers to include
      broken copies of cmake.  So if the error above this refers to some file
      path to a cmake file inside a python or anaconda or miniconda path then you
      should delete that broken copy of cmake from your computer.
  
      Instead, please get an official copy of cmake from one of these known good
      sources of an official cmake:
          - cmake.org (this is how windows users should get cmake)
          - apt install cmake (for Ubuntu or Debian based systems)
          - yum install cmake (for Redhat or CenOS based systems)
  
      On a linux machine you can run `which cmake` to see what cmake you are
      act

In [6]:
%pip install dlib

  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build dlib
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [41 lines of output]
  running bdist_wheel
  running build
  running build_ext
  
  
                     CMake is not installed on your system!
  
      Or it is possible some broken copy of cmake is installed on your system.
      It is unfortunately very common for python package managers to include
      broken copies of cmake.  So if the error above this refers to some file
      path to a cmake file inside a python or anaconda or miniconda path then you
      should delete that broken copy of cmake from your computer.
  
      Instead, please get an official copy of cmake from one of these known good
      sources of an official cmake:
          - cmake.org (this is how windows users should get cmake)
          - apt install cmake (for Ubuntu or Debian based systems)
          - yum install cmake (for Redhat or CenOS based systems)
  
      On a linux machine you can run `which cmake` to see what cmake you are
      act

In [2]:
# Cell 2: Imports
import cv2
import face_recognition
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'face_recognition'

In [3]:
# ID / Passport Tampering Detector (Single Image, CPU)

import os, io
import numpy as np
import cv2
import face_recognition
import matplotlib.pyplot as plt
from PIL import Image, ImageChops, ImageEnhance

# ---------------------------
# Utility functions
# ---------------------------
def load_image_cv(path):
    img = cv2.imread(path)  # BGR
    if img is None:
        raise FileNotFoundError(f"Cannot load image: {path}")
    return img

def pil_from_cv(cv_img):
    return Image.fromarray(cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB))

# ---------------------------
# Error Level Analysis (ELA)
# ---------------------------
def ela_image_cv(cv_img, quality=90, scale=15):
    pil_orig = pil_from_cv(cv_img).convert("RGB")
    bio = io.BytesIO()
    pil_orig.save(bio, format="JPEG", quality=quality)
    bio.seek(0)
    pil_comp = Image.open(bio).convert("RGB")
    diff = ImageChops.difference(pil_orig, pil_comp)
    extrema = diff.getextrema()
    max_diff = max([e[1] for e in extrema]) if extrema else 1
    if max_diff == 0: max_diff = 1
    factor = scale * 255.0 / max_diff
    ela = ImageEnhance.Brightness(diff).enhance(factor)
    ela_arr = np.asarray(ela.convert('L')).astype(np.float32) / 255.0
    return ela_arr

# ---------------------------
# Blur / noise inconsistency
# ---------------------------
def laplacian_variance_gray(gray):
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    return float(lap.var())

# ---------------------------
# HSV statistics
# ---------------------------
def hsv_stats(cv_img):
    hsv = cv2.cvtColor(cv_img, cv2.COLOR_BGR2HSV).astype(np.float32)
    return hsv[...,0].mean(), hsv[...,1].mean(), hsv[...,2].mean()

# ---------------------------
# ORB copy-move detection
# ---------------------------
def copy_move_detection(cv_img, min_good=10):
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1500)
    kps, desc = orb.detectAndCompute(gray, None)
    if desc is None or len(kps) < 20:
        return 0.0, 0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.knnMatch(desc, desc, k=2)
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            if m.queryIdx != m.trainIdx:  # avoid same point
                good.append(m)
    return min(1.0, len(good) / 200.0), len(good)

# ---------------------------
# Face embedding extraction (to catch swaps/fakes)
# ---------------------------
def face_embedding_check(cv_img):
    rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(rgb)
    return len(encodings)

# ---------------------------
# Main tampering detector
# ---------------------------
def analyze_id_tampering(image_path, visualize=True):
    img = load_image_cv(image_path)

    # 1) ELA
    ela_map = ela_image_cv(img)
    ela_score = float(np.mean(ela_map))

    # 2) Blur / sharpness
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sharpness = laplacian_variance_gray(gray)

    # 3) Color stats
    h_mean, s_mean, v_mean = hsv_stats(img)

    # 4) Copy-move detection
    copy_score, copy_matches = copy_move_detection(img)

    # 5) Face embedding check
    faces_detected = face_embedding_check(img)
    embedding_missing = faces_detected == 0

    # Weighted tamper score
    tamper_score = (
        0.4 * ela_score +
        0.2 * (1.0 if sharpness < 100 else 0.0) +  # too blurry = suspicious
        0.2 * copy_score +
        0.2 * (1.0 if embedding_missing else 0.0)
    )
    tamper_score = max(0.0, min(1.0, tamper_score))

    result = {
        "image_path": image_path,
        "ela_score": ela_score,
        "sharpness": sharpness,
        "copy_move_score": copy_score,
        "copy_matches": copy_matches,
        "faces_detected": faces_detected,
        "tamper_score": tamper_score,
        "tampered": tamper_score > 0.5  # threshold
    }

    if visualize:
        fig, axs = plt.subplots(1,2, figsize=(10,5))
        axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[0].set_title("Input Document")
        axs[0].axis("off")
        axs[1].imshow(ela_map, cmap="inferno")
        axs[1].set_title("ELA Map")
        axs[1].axis("off")
        plt.show()

    return result


ModuleNotFoundError: No module named 'face_recognition'